In [20]:
# !pip install shareplum
# !pip install sharepy
# !pip install Office365-REST-Python-Client
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

from shareplum import Site
from shareplum import Office365
from shareplum.site import Version
import csv

In [21]:
import pandas as pd, numpy as np, matplotlib as plt
import re, os, time, datetime, io

In [22]:
# initialize pandas dataframe 'proj_filenames' for the data

proj_filenames = pd.DataFrame(columns=["name", "ext", "project", "code", "year", "month", "type"])

In [23]:
authcookie = Office365('https://feapc.sharepoint.com/', username='jhoward@feapc.com', password='FEAJustHow!333').GetCookies()
site = Site('https://feapc.sharepoint.com/', version=Version.v2016, authcookie=authcookie)

In [5]:
# get the initial (root level) list of folder contents
active_dir = 'FEA Projects'
folderlist = site.list(active_dir).GetListItems(fields=['URL Path'])

In [6]:
# Function to clean the folderlist URL Path into a string equalling the actual folder name
def stripjunk(folderlist):
    
    # turn the list of dictionaries into a list of strings
    for n in range(0,len(folderlist)):
        folderlist[n] = str(folderlist[n].values())
    
    # strip out the initial junk characters
    for n in range(0,len(folderlist)):
        folderlist[n] = folderlist[n].strip("dict_values([\'")

    # strip out the end junk characters
    for n in range(0,len(folderlist)):
        folderlist[n] = folderlist[n].strip('\'])')

    # strip leading numeric chars with regex
    for n in range(0, len(folderlist)):
        while folderlist[n][0].isdigit():
            folderlist[n] = folderlist[n][1::]

    # strip leading numeric chars with regex
    for n in range(0, len(folderlist)):
        folderlist[n] = folderlist[n].strip(';#FEA Projects/')

In [7]:
# apply function to list
stripjunk(folderlist)

In [8]:
# quick cleanup of the list.
folderlist.remove('NPS FY19 PS')
folderlist.sort()

In [9]:
# initialize dataframes for new folders to be added
# initial entries for the root level folders
new_folders = pd.DataFrame({'name': folderlist, 'ext': "n/a", 'project': 'n/a', 'code': "n/a",\
                            'year': folderlist, 'month': "n/a", 'type': 'folder', 'path': '/'})

# join initial folder list into the main dataframe
proj_filenames = pd.concat([proj_filenames, new_folders], axis=0, join='outer')

In [10]:
def add_files(folderlist):
    filenames, exts, projects, codes, years, months, types, paths = ([] for i in range(8))

    for x in range(0, len(folderlist)):
        folderpath = site.Folder(active_dir + '/' + folderlist[x] + '/')

        if len(folderpath.files) > 0:
            newfile = folderpath.files[0]['Name']
            if newfile[0].isnumeric():
                months.append(newfile[x][0:2])
                years.append(newfile[x][3:7])
                codes.append(newfile[8:14])        
                filename = newfile[15::]
            else:
                months = "unknown"
                years = "unknown"
                codes = "unknown"
                filename = newfile

            filename, fileext = filename.split(sep = ".", maxsplit=1)

            if len(fileext) > 5:
                print(f'WARNING: The file extension {filext} seems a bit long. Is this correct?')
            else:
                projects.append(folderlist[x])
                filenames.append(filename)
                exts.append(fileext)
                paths.append("/" + folderlist[x] + "/")
                types.append("file")
        else:
            pass

        # initialize a dataframe for new files
        #new_files = pd.DataFrame(columns=({"name": name, "ext": ext, "project": path, "code": code,\
                                 # "year": year, "month": month, "type": "file"}))


    new_files = pd.DataFrame(
        {'name': filenames,
         'ext': exts,
         'project': projects,
         'code': codes,
         'year': years,     
         'month': months,
         'type': types,
         'path': paths     
        })

    return new_files

In [11]:
new_files = add_files(folderlist)

In [12]:
proj_filenames = pd.concat([proj_filenames, new_files], ignore_index=True)

In [13]:
proj_filenames

,name,ext,project,code,year,month,type,path
0,2006,n/a,n/a,n/a,2006,n/a,folder,/
1,2007,n/a,n/a,n/a,2007,n/a,folder,/
2,2008,n/a,n/a,n/a,2008,n/a,folder,/
3,2009,n/a,n/a,n/a,2009,n/a,folder,/
4,2010,n/a,n/a,n/a,2010,n/a,folder,/
5,2011,n/a,n/a,n/a,2011,n/a,folder,/
6,2012,n/a,n/a,n/a,2012,n/a,folder,/
7,2013,n/a,n/a,n/a,2013,n/a,folder,/
8,2014,n/a,n/a,n/a,2014,n/a,folder,/
9,2015,n/a,n/a,n/a,2015,n/a,folder,/


In [14]:
# save the root folders as it's own list.  
# the variable will be re-set as we go into deeper subfolders
root_folders = folderlist

In [15]:
def add_folders(folderlist):
    
    for i in range(0, len(root_folders)):
        curr_year = root_folders[i]
        active_dir = "FEA Projects/" + curr_year
        folderpath = site.Folder(active_dir + '/')
        subfolders = folderpath.folders
    #    print(f"{curr_year}: {subfolders}")
        for i in range(0, len(subfolders)):
            if len(folderpath.folders) > 0:
                foldernames, exts, projects, codes, years, months, types, paths = ([] for i in range(8))
                foldername = subfolders[i]

                if foldername[0].isnumeric() and foldername[2] == ".":
                    months.append(foldername[0:2])
                    years.append(foldername[3:7])
                    codes.append(foldername[8:14])
                    foldername = foldername[15::]
                else:
                    months.append("unspecified")
                    years.append(curr_year)
                    codes.append("unknown")

                projects.append(subfolders[i])
                foldernames.append(foldername)
                exts.append("n/a")
                paths.append(active_dir + "/")
                types.append("folder")

                new_folders = pd.DataFrame(
                    {'name': foldernames,
                     'ext': exts,
                     'project': projects,
                     'code': codes,
                     'year': years,     
                     'month': months,
                     'type': types,
                     'path': paths
                    })

            else:
                pass
    
    return new_folders

SyntaxError: expected ':' (3453043116.py, line 1)

In [ ]:
 # join the list of new folders into the dataframe
proj_filenames = pd.concat([proj_filenames, new_folders], axis=0, join='outer', ignore_index=True)

In [ ]:
proj_filenames.sample(12)

In [ ]:
pathlist = []
for x in range(0, len(subfolders)):
    folderpath = site.Folder('/' + active_dir + '/' + subfolders[x] + '/')
    pathlist.append(folderpath.folder_name)
print(pathlist)

In [ ]:
folderlist = []
for x in pathlist:
    x = x.strip(active_dir)
    folderlist.append(x)
print(folderlist)

In [ ]:
new_folders = pd.DataFrame({'name': folderlist, 'ext': "n/a", 'project': folderlist,\
                            'code': "n/a", 'year': curr_year, 'month': "not specified",\
                            'type': 'folder', 'path': pathlist})

# join the list of new folders into the dataframe
proj_filenames = pd.concat([proj_filenames, new_folders], axis=0, join='outer')

In [ ]:
proj_filenames

In [ ]:
# get the list of subfolder contents

site.list(folderpath).GetListItems()

# filenames, exts, projects, codes, years, months, types, paths = ([] for i in range(8))

# for x in range(0, len(folderlist)):
#     sub_path = site.Folder(active_dir + '/' + folderlist[x] + '/')
#     folderpath = site.Folder(active_dir + '/' + folderlist[x] + '/')
#     if len(folderpath.files) > 0:
#             newfile = folderpath.files[0]['Name']
#             print(newfile)
            
            
            
#         if len(folderpath.files) > 0:
#             newfile = folderpath.files[0]['Name']
#             if newfile[0].isnumeric():
#                 months.append(newfile[x][0:2])
#                 years.append(newfile[x][3:7])
#                 codes.append(newfile[8:14])        
#                 filename = newfile[15::]
#             else:
#                 months = "unknown"
#                 years = "unknown"
#                 codes = "unknown"
#                 filename = newfile

In [ ]:
active_dir

In [ ]:
## not actually needed for filenames - but will want to adapt this for folders.

# newfile = folder.files[0]['Name']
# month = newfile[0:2]
# year = newfile[3:7]
# code = newfile[8:13]
# newfile = newfile[14::]
# filename, fileext = newfile.split(sep = ".", maxsplit=1)
# #print(f'{filename}')
# # print(f'Month: {month}, Year: {year}, Code: {code}, File: {newfile}')

In [ ]:
def add_subfolders(folderpath)

folderpath = site.Folder(active_dir + "/" + folderlist[0])

# split subfolders list into two lists, codes and names
codes = []
names = []
months = []
years = []
paths = []

for n in range(0, len(subfolders)):
    codes.append(subfolders[n][8:14])
    newnametemp = re.sub("-|\(|\)", "", subfolders[n][15::])
    names.append(newnametemp.strip())
    months.append(subfolders[n][0:2])
    years.append(subfolders[n][3:7])
    paths.append(f'/{subfolders[n][3:7]}/')

# render as a dataframe
subfolder_df = pd.DataFrame(
    {'name': names, 'type': "folder", 'project': codes, 'year': years, 'month': months,\
     'path': paths}
)

# join the list of new folders into the dataframe
proj_filenames = pd.concat([proj_filenames, subfolder_df], axis=0, join='outer')

In [ ]:

# Figure out how to record year and pass as part of a path to strip from string
#         newfile.strip()
#         filelist.append(newfile[0]['ServerRelativeUrl'])

print(f'Month: {month}, Year: {year}')

In [ ]:
# getting filenames in a directory and add to a list
testlist = []
for folder in folderlist:
    folderpath = site.Folder(active_dir + "/" + folder)
    testlist.append(folderpath.files)

for x in range(0, len(testlist)):
    if len(testlist[x]) > 0:
        teststring = testlist[x][0]['ServerRelativeUrl']
        print(teststring)
        
# = testlist[x][0]['ServerRelativeUrl']
# testlist

In [ ]:
# get the first subfolder
folder = site.Folder(active_dir + "/" + folderlist[0])

# later add this to a new function that calls the first one
# or at least a while loop

In [ ]:
# a list of all folder names

subfolders = folder.folders

In [ ]:
proj_filenames.tail(15)

In [ ]:
initial_folderlist = folderlist
active_dir = active_dir + "/" + folderlist[0]
folderlist = site.list(active_dir).GetListItems(fields=['URL Path'])
folderlist

In [ ]:
foldernames = []
for n in range(0,len(folderlist)):
    # strip out the junk
    rawtxt = str(folderlist[n].values())
    rawtxt = rawtxt.strip('dict_values')
    rawtxt = rawtxt.strip('Projec')
    rawtxt = rawtxt.strip('\'])')
    rawtxt = rawtxt.strip('([\'')
    
    foldernames.append(rawtxt)

In [ ]:
regex.regex()

In [ ]:
foldernames[0]

In [ ]:
folder = site.Folder('/FEA Projects/2022')
allfiles = folder.files
demofile = allfiles[0]
print(demofile['Name'])

In [ ]:
type(folder)

In [ ]:
type(demofile)

In [ ]:
# function to take a year range as input and write rows to the dataframe from those years
# 


In [ ]:
# 

In [ ]:
#

In [ ]:
#

In [ ]:
# from https://stackoverflow.com/questions/50531425/get-list-of-files-in-a-sharepoint-directory-using-python
# may use later

# sp_site = 'feapc.sharepoint.com'
# relative_url = "/FEA%20Projects/Forms/"

# client_credentials = ClientCredential(credentials['client_id'], credentials['client_secret'])
# ctx = ClientContext(sp_site).with_credentials(client_credentials)
# libraryRoot = ctx.web.get_folder_by_server_relative_path(relative_url)
# ctx.load(libraryRoot)
# ctx.execute_query()

# #if you want to get the folders within <sub_folder> 
# folders = libraryRoot.folders
# ctx.load(folders)
# ctx.execute_query()
# for myfolder in folders:
#     print("Folder name: {0}".format(myfolder.properties["ServerRelativeUrl"]))

# #if you want to get the files in the folder        
# files = libraryRoot.files
# ctx.load(files)
# ctx.execute_query()

# #create a dataframe of the important file properties for me for each file in the folder
# df_files = pd.DataFrame(columns = ['Name', 'ServerRelativeUrl', 'TimeLastModified', 'ModTime'])
# for myfile in files:
#     #use mod_time to get in better date format
#     mod_time = datetime.datetime.strptime(myfile.properties['TimeLastModified'], '%Y-%m-%dT%H:%M:%SZ')  
#     #create a dict of all of the info to add into dataframe and then append to dataframe
#     dict = {'Name': myfile.properties['Name'], 'ServerRelativeUrl': myfile.properties['ServerRelativeUrl'], 'TimeLastModified': myfile.properties['TimeLastModified'], 'ModTime': mod_time}
#     df_files = df_files.append(dict, ignore_index= True )

#     #print statements if needed
#     # print("File name: {0}".format(myfile.properties["Name"]))
#     # print("File link: {0}".format(myfile.properties["ServerRelativeUrl"]))
#     # print("File last modified: {0}".format(myfile.properties["TimeLastModified"]))
# #get index of the most recently modified file and the ServerRelativeUrl associated with that index
# newest_index = df_files['ModTime'].idxmax()
# newest_file_url = df_files.iloc[newest_index]['ServerRelativeUrl']

# # Get Excel File by newest_file_url identified above
# response= File.open_binary(ctx, newest_file_url)
#     # save data to BytesIO stream
# bytes_file_obj = io.BytesIO()
# bytes_file_obj.write(response.content)
# bytes_file_obj.seek(0)  # set file object to start
#     # load Excel file from BytesIO stream
# df = pd.read_excel(bytes_file_obj, sheet_name='Sheet1', header= 0)